# 'Exact' exchange in CP2K

vanilla implementation in CP2k - there are a bunch of other methods more or less experimental...

Main papers are: 

Minimum Image Convention methodology and main details: [J. Chem. Phys. 128, 214104 (2008)](https://scholar.archive.org/work/2wkry3b47rg3bljth7xb7r36cy/access/wayback/http://www.zora.uzh.ch/id/eprint/3154/4/RP_Guidon_JCP_2008PPV.pdf)

and

Truncated Coulomb interaction and better periodic implementation: [J. Chem. Theory Comput. 2009, 5, 11, 3010–3021](https://pubs.acs.org/doi/pdf/10.1021/ct900494g)

code is available on Github in files with hfx_ prefix e.g.

https://github.com/cp2k/cp2k/blob/master/src/hfx_energy_potential.F

all in Fortran 2008 standard:
https://www.cp2k.org/dev:codingconventions

# Integral evaluation

Integrals are evaluated using the libint library from Valeev (PNNL) (https://github.com/evaleev/libint).

$$
(\mu \nu \lvert \lambda \sigma)_g = \int d\mathbf{r}_1 d\mathbf{r}_2 \mu(\mathbf{r}_1) \nu(\mathbf{r}_1) g(r_{12}) \lambda(\mathbf{r}_2) \sigma(\mathbf{r}_2)
$$

For non-periodic calculation we can use the stand integrals from the libint library, e.g. $g(r_{12}) = \frac{1}{r_{12}}$. 

#### Truncated potential

For periodic systems with a standard global hybrid functional typically a truncated interaction is used:

$$
g_{TC}(r_{12}) = \begin{cases}
\frac{1}{r_{12}},  r_{12} \le R_c\\
0, r_{12} \gt R_c\
\end{cases}
$$

this allows a $\Gamma$ point approximation to the Hartree-Fock energy to be given as:

$$
E_x^\Gamma = - \frac{1}{2} \sum_{ij} \int d\mathbf{r}_1 d\mathbf{r}_2 \phi_i^0 (\mathbf{r}_1) \phi_j^0 (\mathbf{r}_1) g(r_{12}) \phi_i^0 (\mathbf{r}_2) \phi_j^0 (\mathbf{r}_2)
$$

and in terms of the atomic basis functions:

$$
E_x^\Gamma = - \frac{1}{2} \sum_{\lambda \sigma \mu \nu } P^{\mu \sigma} P^{\nu \lambda} \sum_{\mathbf{abc}} (\mu \nu^{\mathbf{a}} \lvert \lambda^{\mathbf{b}} \sigma ^{\mathbf{b+c}})_{g_{TC}}
$$
where $\mathbf{abc}$ are translations of the unit cell.

At the moment the individual terms in the sum are not stored just the sum over the cell vectors...

Details of the evaluation of the truncated potential using libint are in the 2009 paper. Ugly but works well...

# Screening

heirarchy of methods: 

Near-field Schwarz screening based on Schwarz equality.

Density matrix screening

# Task lists

Setup and loadbalancing are handled using a talk list method. 

Costs for tasks is quite well estimated. For MD the costs can be updated based on previous MD steps.

Load balancing methods are not ideal scaling and turned off for largest systems...

# Parallelisation

Density and KS matrices are replicated on each MPI process. This makes calculation and contraction of integrals pretty much embarrassingly parallel.

For larger systems need to use OpenMP/MPI joint parallelisation to allow eris to be stored in memory.

# Integral compression

Integrals are compressed after summing over images to enable in-core operation.
Details in [J. Chem. Phys. 128, 214104 (2008)](https://scholar.archive.org/work/2wkry3b47rg3bljth7xb7r36cy/access/wayback/http://www.zora.uzh.ch/id/eprint/3154/4/RP_Guidon_JCP_2008PPV.pdf).